In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
print(torch.cuda.is_available())  # Should print True if GPU is enabled

True


In [1]:
!pip install ultralytics
from ultralytics import YOLO



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.5/921.5 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get install unrar

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [ ]:
!unrar x /content/drive/MyDrive/YOLOv8_data_arman.rar /content/dataset/


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/drive/MyDrive/YOLOv8_data_arman.rar


Would you like to replace the existing file /content/dataset/train/images/dayClip1--00000.jpg
130347 bytes, modified on 2019-09-27 17:57
with a new one
130347 bytes, modified on 2019-09-27 17:57

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit E

All OK


In [2]:
model = YOLO('yolov8n.pt')

100%|██████████| 6.25M/6.25M [00:00<00:00, 171MB/s]


In [ ]:
import torch
import torch.nn as nn
from ultralytics import YOLO

# BiFPN Neck
class BiFPN(nn.Module):
    """Bidirectional Feature Pyramid Network for better feature fusion"""
    def __init__(self, in_channels=[64, 128, 256, 512, 1024], out_channels=256):
        super(BiFPN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels[-1], out_channels, kernel_size=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x[-1])  # Use highest-level feature map
        x = self.conv2(x)
        return self.relu(x)

# YOLOv8n with BiFPN
class YOLOv8nBiFPN(nn.Module):
    """YOLOv8n with BiFPN Neck"""
    def __init__(self):
        super(YOLOv8nBiFPN, self).__init__()
        self.neck = BiFPN()

    def forward(self, x):
        x = self.neck(x)
        return x

# Training Function
def train_yolov8n():
    model = YOLO("yolov8n.yaml")  # Load YOLOv8n config
    model.model.neck = BiFPN()  # Replace neck with BiFPN
    model.train(data='/content/dataset/dataset.yaml', epochs=40,batch=96, imgsz=640,project='/content/drive/MyDrive/arman_tests',name='improvedYOLOv8n2_40_640_round1')  # Train model

# Run Training
if __name__ == "__main__":
    train_yolov8n()


Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/dataset/dataset.yaml, epochs=40, time=None, patience=100, batch=96, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/arman_tests, name=improvedYOLOv8n2_40_640_round12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, 

train: Scanning /content/dataset/train/labels... 14034 images, 1259 backgrounds, 0 corrupt: 100%|██████████| 14034/14034 [00:11<00:00, 1220.77it/s]


train: New cache created: /content/dataset/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.4' (you have '2.0.3'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
val: Scanning /content/dataset/valid/labels... 468 images, 57 backgrounds, 0 corrupt: 100%|██████████| 468/468 [00:00<00:00, 730.53it/s]


val: New cache created: /content/dataset/valid/labels.cache
Plotting labels to /content/drive/MyDrive/arman_tests/improvedYOLOv8n2_40_640_round12/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.00075), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to /content/drive/MyDrive/arman_tests/improvedYOLOv8n2_40_640_round12
Starting training for 40 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      40/40      11.9G     0.4399     0.2647      0.815         38        640: 100%|██████████| 147/147 [04:52<00:00,  1.99s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:06<00:00,  2.03s/it]

                   all        468       1152      0.869      0.915      0.923      0.576



40 epochs completed in 3.440 hours.
Optimizer stripped from /content/drive/MyDrive/arman_tests/improvedYOLOv8n2_40_640_round12/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/arman_tests/improvedYOLOv8n2_40_640_round12/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/arman_tests/improvedYOLOv8n2_40_640_round12/weights/best.pt...
Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n summary (fused): 168 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:10<00:00,  3.62s/it]


                   all        468       1152      0.868      0.914      0.923      0.575
                    go        215        426       0.81      0.899      0.909      0.579
                  stop        196        412      0.895      0.922      0.951      0.576
              stopLeft        314        314        0.9       0.92      0.909      0.569
Speed: 0.2ms preprocess, 1.7ms inference, 0.0ms loss, 4.6ms postprocess per image
Results saved to /content/drive/MyDrive/arman_tests/improvedYOLOv8n2_40_640_round12
